In [142]:
import pandas as pd
import numpy as np
from datetime import datetime
import pickle
import concurrent.futures 
from fuzzywuzzy import fuzz, process
from difflib import SequenceMatcher
from os import walk
from bs4 import BeautifulSoup
import requests
import time
import random
pd.set_option('display.max_columns', 100)
import statistics
import warnings
warnings.filterwarnings("ignore")
from scipy.stats import skew, kurtosis


In [143]:
# function to determine the winner based on the score difference
def winner(x):
    if x == 0:  # no difference, draw
        return 0
    elif x > 0:  # positive difference, home team wins
        return 1
    else:  # negative difference, away team wins
        return 2

# function to scrape the score from a webpage and determine the winner
def get_winner(url):
    try:
        page = requests.get(url)
        # pause for interval between 0 and 1 seconds -> got banned before
        time.sleep(random.randint(0, 1))
        soup = BeautifulSoup(page.content, 'html.parser')
        # find the score on the page and calculate the difference
        score = soup.find_all("span", {"class": "css-uzl3lr"})[0].text.split(' - ')
        score = int(score[0]) - int(score[1])
        # determine the winner based on the score difference
        score = winner(score)
        return score
    except:
        # if error, happens when game was abondoned or was never played
        return 'remove'

# function to calculate the similarity between two strings
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

# function to find the most similar name from a list to a given name
def getname(name, l):
    if len(l) == 0:  # if the list is empty, return an empty string
        return ''
    highest = process.extractOne(name,l)
    return highest[0]


In [144]:
o = '/Users/idotzhori/Library/CloudStorage/OneDrive-NortheasternUniversity/Soccer Analysis'
n = '/Users/idotzhori/Library/CloudStorage/OneDrive-NortheasternUniversity/Buffalo Bets/Data'
old_f = next(walk(o), (None, None, []))[2]
old_f = [f'{o}/{f}' for f in old_f if '2022' in f and '_data' in f and 'dict' not in f and 'all' not in f]
new_f = next(walk(n), (None, None, []))[2]
new_f = [f'{n}/{f}' for f in new_f]
file_names = old_f + new_f

In [145]:
all_data = []

for index, f in enumerate(file_names):
    open_file = open(f, "rb")
    # load the pickled data from the file
    d = pickle.load(open_file)
    # append the loaded data to the all_data list
    all_data.append(d)
    open_file.close()
    # print a progress update every 15 files
    if (index + 1) % 15 == 0:
        print("Processing iteration", index + 1)

# get the total number of days of data collected
total_days = len(all_data)
print("\nFinal data processing completed. Obtained data for", total_days, "days.")

# flatten the list of lists into a single list
all_data = [i for j in all_data for i in j]

Processing iteration 15
Processing iteration 30
Processing iteration 45
Processing iteration 60
Processing iteration 75
Processing iteration 90
Processing iteration 105
Processing iteration 120

Final data processing completed. Obtained data for 123 days.


In [146]:
len(all_data)

3523

In [185]:
res = []

for idx, dic in enumerate(all_data):
    try:
        e_dict = {}
        for name, tup in dic.items():
            # standardize the key name by replacing spaces with underscores and converting to lowercase
            name = name.replace(' ','_').lower()
            # I collected data 2 ways, so this is to account for the difference
            try:
                # if the value is a list of two elements and the first element is a string or float
                # calculate the difference between the two elements
                if (len(tup) == 2) and (isinstance(tup, list)) and (isinstance(tup[0], str) or isinstance(tup[0], float)):
                    e_dict[f'{name}_diff'] = float(tup[0]) - float(tup[1])
                else:
                    # if the conditions are not met, just store the value as is
                    e_dict[name] = tup
            except:
                # if an exception is raised, just store the value as is
                e_dict[name] = tup        
        # data cleanup for earlier attebmpts of data scraping
        if 'bet' in e_dict:
            # calculate the sum of odds and delete the 'bet' key-value pair
            e_dict['odds_sum'] = e_dict['bet']['odds_checker_sum']
            del e_dict['bet']
        # rename the 'odds' key to 'median_odds'
        e_dict['median_odds'] = e_dict.pop('odds')
        # if the 'median_odds' dictionary has three elements (draw, home away)
        if len(e_dict['median_odds']) == 3:
            # calculate and store the standard deviation of the 'full_odds' values
            # removes the two largest and smallest odds (when oddscraper didn't load)
            e_dict['std_odds'] = dict(zip(list(e_dict['median_odds'].keys()),
                                          [np.std(sorted(x)[2:-2]) for x in e_dict['full_odds']]))
            # calculate skew and kurtosis
            e_dict['skew_odds'] = dict(zip(list(e_dict['median_odds'].keys()),
                                           [skew(sorted(x)[2:-2]) for x in e_dict['full_odds']]))
            e_dict['kurtosis_odds'] = dict(zip(list(e_dict['median_odds'].keys()),
                                               [kurtosis(sorted(x)[2:-2]) for x in e_dict['full_odds']]))
            # calculate and store the 75th percentile of the 'full_odds' values
            e_dict['75_odds'] = dict(zip(list(e_dict['median_odds'].keys()),
                                         [np.percentile(sorted(x), 75) for x in e_dict['full_odds']]))
            # range of each odds
            try:
                e_dict['range_odds'] = dict(zip(list(e_dict['median_odds'].keys()),
                                [max(sorted(x)[2:-2]) - min(sorted(x)[2:-2]) for x in e_dict['full_odds']]))
            except:
                e_dict['range_odds'] = dict(zip(list(e_dict['median_odds'].keys()),
                                [max(sorted(x)) - min(sorted(x)) for x in e_dict['full_odds']]))
            
            # mean odds, like median odds
            try:
                e_dict['mean_odds'] = dict(zip(list(e_dict['median_odds'].keys()),
                                          [np.mean(sorted(x)[2:-2]) for x in e_dict['full_odds']]))
            except:
                e_dict['mean_odds'] = dict(zip(list(e_dict['median_odds'].keys()),
                                          [np.mean(sorted(x)) for x in e_dict['full_odds']]))

        # append the processed dictionary to the result list
        res.append(e_dict)
#         elif ((len(e_dict['median_odds']) != 3) & (similar("".join(sorted(e_dict['match_name'])), "".join(sorted(e_dict['api_odds']['best_match']))) > 0.9)
#            & ('api_odds' in e_dict)):
#             odds = [i for j in list(e_dict['api_odds'].values())[:-1] for i in j]
#             odds = np.array(odds).T.tolist()
#             names = e_dict['match_name'].split('-vs-')
#             names = [names[0]] + ['draw'] + [names[1]]
#             e_dict['median_odds'] = dict(zip(names, [statistics.median(o[1:-1]) for o in odds]))
#             e_dict['std_odds'] = dict(zip(names, [np.std(sorted(x)[1:-1]) for x in odds]))
#             odds_sum = np.sum([1/o for o in e_dict['median_odds'].values()])
#             e_dict['odds_sum'] = odds_sum
#         else:
#             pass
#         if (idx + 1) % 500 == 0:
#             print(f"Processed {idx + 1} games...")
    except Exception as e:
        dic['skew_odds'] = dict(zip(list(dic['median_odds'].keys()),
                                           [skew(sorted(x)[2:-2]) for x in dic['full_odds']]))
        dic['kurtosis_odds'] = dict(zip(list(dic['median_odds'].keys()),
                                               [kurtosis(sorted(x)[2:-2]) for x in dic['full_odds']]))
            # calculate and store the 75th percentile of the 'full_odds' values
        dic['75_odds'] = dict(zip(list(dic['median_odds'].keys()),
                                         [np.percentile(sorted(x), 75) for x in dic['full_odds']]))
            # range of each odds
        try:
            dic['range_odds'] = dict(zip(list(dic['median_odds'].keys()),
                                [max(sorted(x)[2:-2]) - min(sorted(x)[2:-2]) for x in dic['full_odds']]))
        except:
            dic['range_odds'] = dict(zip(list(dic['median_odds'].keys()),
                                [max(sorted(x)) - min(sorted(x)) for x in dic['full_odds']]))
            
            # mean odds, like median odds
        try:
            dic['mean_odds'] = dict(zip(list(dic['median_odds'].keys()),
                                          [np.mean(sorted(x)[2:-2]) for x in dic['full_odds']]))
        except:
            dic['mean_odds'] = dict(zip(list(dic['median_odds'].keys()),
                                          [np.mean(sorted(x)) for x in dic['full_odds']]))

        res.append(dic)
        
print("Done!")

Done!


In [186]:
len(res)

3523

In [187]:
df = pd.DataFrame(res)
len_rows = len(df)
print(f'Initial dataframe size: {len_rows}')
df = df[df['half_time'] == 'Half-Time']
print(f'Rows filtered due to wrong timing: {np.abs(len(df) - len_rows)}')

# didn't collect the odds data
len_rows = len(df)
df = df[(df['median_odds'] != '') & (df['median_odds'] != {}) & (df['median_odds'].notna())]
print(f'Rows filtered due to scraping issue": {np.abs(len(df) - len_rows)}')

# only collected one or two odds (eg. only away and home odds, no draw)
len_rows = len(df)
df = df[df['median_odds'].map(len) == 3]
print(f'Rows filtered due to oddschecker bugs": {np.abs(len(df) - len_rows)}')

# odds_sum < 1.03 if there is a clear clear winner (5-0 up at half)
# in which case there is no value in betting

# odds_sum > 1.11 usually means there is a data issue with oddschecker

len_rows = len(df)
df = df[(df['odds_sum'] > 1.03) & (df['odds_sum'] < 1.11)]
print(f'Rows filtered due to non-agreeing odds data": {np.abs(len(df) - len_rows)}')

# not enough bookies are offering odds, means there is no value or data issue
len_rows = len(df)
df['full_odds_min'] = [min([len(x) for x in l]) for l in df['full_odds']]
df = df[df['full_odds_min'] > 6]
print(f'Rows filtered due to insufficient odds data": {np.abs(len(df) - len_rows)}')

df = df.reset_index(drop = True)

# need to reorder odds data to draw - home - away since oddschecker is not standardized in the order
ccc = 0
# first get the true home away teams from the url link
df['o_names'] = df['odds_url'].apply(lambda x: ['draw'] + x.split('/')[-2].split('-v-'))
for name in ['std','median','75','skew','kurtosis','range','mean']:
    r = []
    for i in df[[f'{name}_odds','o_names']].to_numpy():
        # get the correct order and reorder the odds to match
        match = [getname(name, list(i[0].keys())) for name in i[1]]
        match = [i[0][key] for key in match]
        r.append(match)
        ccc+=1
    df[f'{name}_odds'] = r

# minimum value of the median odds is the bookie favorite
df['odds_predict'] = df['median_odds'].apply(lambda x: np.argmin(x))


ordo_df = pd.DataFrame(df['median_odds'].tolist(), columns=['med_0', 'med_1', 'med_2'])
std_o = pd.DataFrame(df['std_odds'].tolist(), columns=['std_0', 'std_1', 'std_2'])


df = pd.concat([df, std_o, ordo_df], axis=1)

# agreement within the 
df['cv_odds'] = df[['med_0','med_1','med_2']].std(axis = 1)/df[['med_0','med_1','med_2']].mean(axis = 1)
df = df.drop(['half_time','o_names', '75_odds', 'median_odds',
             'std_odds', 'full_odds','match_name', 'full_odds_min','league_name',
             'xg_first_half_diff','xg_second_half_diff','api_odds'], axis = 1)
print(df.shape)

Initial dataframe size: 3523
Rows filtered due to wrong timing: 424
Rows filtered due to scraping issue": 87
Rows filtered due to oddschecker bugs": 185
Rows filtered due to non-agreeing odds data": 112
Rows filtered due to insufficient odds data": 50
std
median
75
skew
kurtosis
range
mean
(2665, 64)


In [188]:
df.iloc[1307][-50:]#['full_odds']

shots_outside_box_diff                                                               3.0
xg_open_play_diff                                                                   0.33
xg_set_play_diff                                                                    0.21
xg_on_target_(xgot)_diff                                                           -0.49
passes_diff                                                                         75.0
own_half_diff                                                                       39.0
opposition_half_diff                                                                41.0
accurate_long_balls_diff                                                             3.0
accurate_long_balls_percentage_diff                                                 0.12
accurate_crosses_diff                                                                4.0
accurate_crosses_percentage_diff                                                    0.04
throws_diff          

In [183]:
df.iloc[1307][-50:]

passes_diff                                                                         75.0
own_half_diff                                                                       39.0
opposition_half_diff                                                                41.0
accurate_long_balls_diff                                                             3.0
accurate_long_balls_percentage_diff                                                 0.12
accurate_crosses_diff                                                                4.0
accurate_crosses_percentage_diff                                                    0.04
throws_diff                                                                          4.0
yellow_cards_diff                                                                   -1.0
red_cards_diff                                                                       0.0
tackles_won_diff                                                                     2.0
tackles_won_percentag

In [8]:
test = df[['url','odds_url']]
test['url'] = test['url'].apply(lambda x: x.split('/')[-1])
test['odds_url'] = test['odds_url'].apply(lambda x: x.split('/')[-2])
test['s'] = test[['url','odds_url']].apply(lambda x: similar(x.url,x.odds_url), axis = 1)
test = test.sort_values(by = 's')
test.iloc[0:60]

,url,odds_url,s
705,burnley-vs-preston-north-end,stoke-v-hull,0.250000
1678,gif-sundsvall-vs-landskrona-bois,servette-v-young-boys,0.264151
1709,bonnyrigg-rose-athletic-vs-stranraer,ross-county-v-celtic,0.285714
775,swansea-city-vs-huddersfield-town,watford-v-bristol-city,0.290909
1344,swansea-city-vs-preston-north-end,watford-v-cardiff-city,0.290909
913,annan-athletic-vs-stenhousemuir,hearts-v-st-johnstone,0.307692
1343,queens-park-rangers-vs-norwich-city,watford-v-cardiff-city,0.315789
911,tranmere-rovers-vs-hartlepool-united,sutton-united-v-crewe,0.315789
2201,arminia-bielefeld-vs-fc-heidenheim,hamburg-v-fc-nurnberg,0.327273
2292,ifk-norrk%C3%B6ping-vs-mj%C3%A4llby,ifk-varnamo-v-kalmar-ff,0.344828


In [7]:
# remove incorrect matches between fotmob data and oddschecker data

remove =  [ 705, 1678, 1709,  775, 1344,  913, 1343,  911, 2201, 2292,  487,
            2026, 2294,  748, 1679, 1511,  469,  139,  462, 1342, 1690, 2394,
             564, 2247, 2324, 1296,  914,  413, 659]

df = df.drop(remove)
df = df.reset_index(drop = True)
df = df.drop(['_diff', '1.00_diff','0.20_diff', '1.17_diff', '0.39_diff'], axis = 1)

In [10]:
scores = []
batch_size = 50
total_batches = len(df['url']) // batch_size + 1

for i in range(0, len(df['url']), batch_size):
    x = list(df['url'][i:i + batch_size])
    current_batch = i // batch_size + 1
    print(f'Processing batch {current_batch} of {total_batches}, total scores gathered: {len(scores)}')
    
    with concurrent.futures.ThreadPoolExecutor() as executor:
        r = executor.map(get_winner, x)
    
    for result in r:
        scores.append(result)
    
    time.sleep(2)

Processing batch 1 of 53, total scores gathered: 0
Processing batch 2 of 53, total scores gathered: 50
Processing batch 3 of 53, total scores gathered: 100
Processing batch 4 of 53, total scores gathered: 150
Processing batch 5 of 53, total scores gathered: 200
Processing batch 6 of 53, total scores gathered: 250
Processing batch 7 of 53, total scores gathered: 300
Processing batch 8 of 53, total scores gathered: 350
Processing batch 9 of 53, total scores gathered: 400
Processing batch 10 of 53, total scores gathered: 450
Processing batch 11 of 53, total scores gathered: 500
Processing batch 12 of 53, total scores gathered: 550
Processing batch 13 of 53, total scores gathered: 600
Processing batch 14 of 53, total scores gathered: 650
Processing batch 15 of 53, total scores gathered: 700
Processing batch 16 of 53, total scores gathered: 750
Processing batch 17 of 53, total scores gathered: 800
Processing batch 18 of 53, total scores gathered: 850
Processing batch 19 of 53, total scores 

In [11]:
len(scores)

2629

In [12]:
df['target'] = scores
df = df.drop(['url','odds_url'], axis = 1)
df = df[df['target'] != 'remove']

In [13]:
df.to_csv('ml_df.csv')